In [6]:
import shutil
import os
import pandas as pd
import librosa
import numpy as np
import soundfile as sf

# Load the CSV file
train_curated_df = pd.read_csv("train_curated.csv")

# Filter the dataframe to get rows with the label "Bicycle_bell"
bicycle_bell_files = train_curated_df[train_curated_df['labels'] == "Bicycle_bell"]

def extract_active_part(y, sr, duration=2.0, threshold=-40.0):
    """
    Extract the active part of an audio signal.
    
    Parameters:
    - y: Audio signal
    - sr: Sample rate
    - duration: Duration of the extracted segment in seconds
    - threshold: Threshold in dB to consider as active
    
    Returns:
    - y_out: Extracted audio segment
    """
    # Compute the short-time energy
    ste = librosa.feature.rms(y=y)[0]
    ste_db = librosa.amplitude_to_db(ste, ref=np.max)
    
    # Find the first frame where energy exceeds the threshold
    active_frames = np.where(ste_db > threshold)[0]
    
    if len(active_frames) == 0:
        # No active part found
        return y[:int(sr * duration)]
    
    start_frame = active_frames[0]
    start_sample = librosa.frames_to_samples(start_frame)
    end_sample = start_sample + int(sr * duration)
    
    # Extract the segment
    y_out = y[start_sample:end_sample]
    
    return y_out

# Source and destination directories
src_dir = "train_curated/"
dest_dir = "Bicycle_bell_sounds/"

# Create the destination directory if it doesn't exist
if not os.path.exists(dest_dir):
    os.makedirs(dest_dir)

for file_name in bicycle_bell_files['fname']:
    # Load the audio file
    file_path = os.path.join(src_dir, file_name)
    y, sr = librosa.load(file_path, sr=None)
    
    # Extract the active part
    y_extracted = extract_active_part(y, sr)
    
    # Save the extracted part to the destination directory
    output_path = os.path.join(dest_dir, file_name)
    sf.write(output_path, y_extracted, sr)

"Files have been processed and saved successfully!"


'Files have been processed and saved successfully!'

In [2]:
! pip install numpy==1.24.0
import numpy as np

print(np.__version__)

1.24.0
